In [2]:
import os
from io import StringIO
import pandas as pd
import numpy as np
import skbio.io
from qiime2 import Artifact

In [10]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()   

NCBI database not present yet (first time used?)
Done. Parsing...


Loading node names...
2488560 names loaded.
297483 synonyms loaded.
Loading nodes...
2488560 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /home/blue_pc/.etetoolkit/taxa.sqlite ...
 2488000 generating entries... 

Inserting synonyms:      20000 


Uploading to /home/blue_pc/.etetoolkit/taxa.sqlite



Inserting synonyms:      295000 

Inserting taxids:       20000  

Inserting taxids:       2485000 

In [3]:
representative_seq = Artifact.load("output/qza/representative_seqs.qza")
representative_seq.export_data("output/taxonomy")

In [ ]:
representative_seq

In [4]:
seq_path = "output/taxonomy/dna-sequences.fasta"
taxa_path = "output/taxonomy/taxa.txt"
dbpath = "~/blast_db/16S_ribosomal_RNA"
cpu_num = os.cpu_count()

In [6]:
!blastn -query $seq_path -out $taxa_path\
-task megablast -db $dbpath -num_threads $cpu_num\
-evalue 1e-10 -best_hit_score_edge 0.01 -best_hit_overhang 0.01\
-outfmt "+7 qseqid staxids" -perc_identity 95 -max_target_seqs 1

In [26]:
blast_cmd.split(" ")

['blastn',
 '-query',
 'output/taxonomy/dna-sequences.fasta',
 '-out',
 'output/taxonomy/taxa.txt-task',
 'megablast',
 '-db',
 '~/blast_db/16S_ribosomal_RNA',
 '-num_threads',
 '24-evalue',
 '1e-10',
 '-best_hit_score_edge',
 '0.01',
 '-best_hit_overhang',
 '0.01-outfmt',
 '"+7',
 'qseqid',
 'staxids"',
 '-perc_identity',
 '95',
 '-max_target_seqs',
 '1']

In [33]:
blast_cmd

'blastn -query output/taxonomy/dna-sequences.fasta -out output/taxonomy/taxa.txt-task megablast -db ~/blast_db/16S_ribosomal_RNA -num_threads 24-evalue 1e-10 -best_hit_score_edge 0.01 -best_hit_overhang 0.01-outfmt "+7 qseqid staxids evalue bitscore" -perc_identity 95 -max_target_seqs 1'

In [39]:
blast_cmd = f"""blastn -query {seq_path} -out {taxa_path} \
-task megablast -db {dbpath} -num_threads {cpu_num} \
-evalue 1e-7 -best_hit_score_edge 0.01 -best_hit_overhang 0.01 \
-outfmt "+7 qseqid staxids" -perc_identity 95 -max_target_seqs 1"""
import subprocess

output = subprocess.run(blast_cmd, shell=True, capture_output=True, text=True)

with open(taxa_path, "r") as f:
    fh = StringIO(f.read())

df = skbio.io.read(fh, into=pd.DataFrame)
df = df.drop_duplicates(["qseqid"])
df = df.set_index("qseqid")
df.to_csv("output/taxonomy/qid_taxid.csv", index=None)


In [41]:
for i in df.index:
    lineage = ncbi.get_lineage(df.loc[i, "staxids"]) # 取得lineage list
    dic = {v: k for k, v in ncbi.get_rank(lineage).items()} # id:rank字典翻轉成rank:id
    df.loc[i, dic.keys()] = dic.values() # 利用rank作為columns新增至df

In [42]:
# 只取出7個層級
rank_list = ["k", "p", "c", "o", "f", "g", "s"]
taxa = df.loc[:, ["superkingdom", "phylum", "class", "order", "family", "genus", "species"]]
taxa.columns = rank_list
taxa = taxa.fillna(-1).astype(np.int64).astype(str)
taxa.head()

,k,p,c,o,f,g,s
qseqid,,,,,,,
75eb829cc54be1d37541bad22e2b104c,2,976,200643,171549,815,909656,310297
aad49886d464a811b3c71c4fff2be6a6,2,976,200643,171549,815,909656,310297
fc4acd2822bc79e4a208ca9e00d17dbe,2,1239,186801,186802,186803,841,301302
0757a94ad3aae3e0b6744a115e0124c5,2,976,200643,171549,815,909656,821
637cffe4d351e63641b7e01fb94e49f9,2,976,200643,171549,815,909656,357276


In [16]:
taxa

,k,p,c,o,f,g,s
qseqid,,,,,,,
75eb829cc54be1d37541bad22e2b104c,2,976,200643,171549,815,909656,310297
aad49886d464a811b3c71c4fff2be6a6,2,976,200643,171549,815,909656,310297
fc4acd2822bc79e4a208ca9e00d17dbe,2,1239,186801,186802,186803,841,301302
0757a94ad3aae3e0b6744a115e0124c5,2,976,200643,171549,815,909656,821
637cffe4d351e63641b7e01fb94e49f9,2,976,200643,171549,815,909656,357276
...,...,...,...,...,...,...,...
12dd6e03198da1408fb926d1418b300b,2,1239,186801,186802,990719,990721,270498
8522df18632156459a01559989fc84c0,2,508458,649775,649776,649777,1434006,651822
bae9de848c8dfff7271302ee748a554f,2,976,200643,171549,2005519,1348911,1099853


In [43]:
# 可以先concat後將species id相同的feature合併
asv = pd.read_csv("output/table/asv.csv", index_col=0)
asv

,10,11,12,13,14,17,18,19,2,20,...,27,29,3,30,4,5,6,7,8,9
75eb829cc54be1d37541bad22e2b104c,0.0,6864.0,3937.0,10355.0,0.0,52431.0,44.0,10142.0,1280.0,13208.0,...,24.0,2.0,18.0,33.0,10272.0,42.0,0.0,2144.0,129.0,5270.0
aad49886d464a811b3c71c4fff2be6a6,38.0,51771.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1307.0,0.0,0.0,0.0,0.0,0.0,563.0,273.0,35268.0
fc4acd2822bc79e4a208ca9e00d17dbe,704.0,517.0,9050.0,100.0,0.0,258.0,3174.0,11954.0,2380.0,8907.0,...,152.0,1867.0,173.0,730.0,757.0,17924.0,34.0,1175.0,2275.0,74.0
0757a94ad3aae3e0b6744a115e0124c5,14413.0,112.0,1224.0,3476.0,0.0,0.0,2237.0,342.0,953.0,2577.0,...,250.0,2218.0,1838.0,568.0,8699.0,13664.0,2498.0,847.0,476.0,206.0
637cffe4d351e63641b7e01fb94e49f9,0.0,4155.0,0.0,0.0,6088.0,7772.0,6150.0,1060.0,57.0,1126.0,...,15495.0,0.0,1104.0,0.0,0.0,0.0,0.0,0.0,549.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bae9de848c8dfff7271302ee748a554f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
576b578b9b1f7993e8ac4c0d02b349b8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1c8d733e97566147a94100b0664bf84b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
6c1e72c947c8b80c7df0a6eb736e09f4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [44]:
taxa

,k,p,c,o,f,g,s
qseqid,,,,,,,
75eb829cc54be1d37541bad22e2b104c,2,976,200643,171549,815,909656,310297
aad49886d464a811b3c71c4fff2be6a6,2,976,200643,171549,815,909656,310297
fc4acd2822bc79e4a208ca9e00d17dbe,2,1239,186801,186802,186803,841,301302
0757a94ad3aae3e0b6744a115e0124c5,2,976,200643,171549,815,909656,821
637cffe4d351e63641b7e01fb94e49f9,2,976,200643,171549,815,909656,357276
...,...,...,...,...,...,...,...
12dd6e03198da1408fb926d1418b300b,2,1239,186801,186802,990719,990721,270498
8522df18632156459a01559989fc84c0,2,508458,649775,649776,649777,1434006,651822
bae9de848c8dfff7271302ee748a554f,2,976,200643,171549,2005519,1348911,1099853


In [45]:
asv

,10,11,12,13,14,17,18,19,2,20,...,27,29,3,30,4,5,6,7,8,9
75eb829cc54be1d37541bad22e2b104c,0.0,6864.0,3937.0,10355.0,0.0,52431.0,44.0,10142.0,1280.0,13208.0,...,24.0,2.0,18.0,33.0,10272.0,42.0,0.0,2144.0,129.0,5270.0
aad49886d464a811b3c71c4fff2be6a6,38.0,51771.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1307.0,0.0,0.0,0.0,0.0,0.0,563.0,273.0,35268.0
fc4acd2822bc79e4a208ca9e00d17dbe,704.0,517.0,9050.0,100.0,0.0,258.0,3174.0,11954.0,2380.0,8907.0,...,152.0,1867.0,173.0,730.0,757.0,17924.0,34.0,1175.0,2275.0,74.0
0757a94ad3aae3e0b6744a115e0124c5,14413.0,112.0,1224.0,3476.0,0.0,0.0,2237.0,342.0,953.0,2577.0,...,250.0,2218.0,1838.0,568.0,8699.0,13664.0,2498.0,847.0,476.0,206.0
637cffe4d351e63641b7e01fb94e49f9,0.0,4155.0,0.0,0.0,6088.0,7772.0,6150.0,1060.0,57.0,1126.0,...,15495.0,0.0,1104.0,0.0,0.0,0.0,0.0,0.0,549.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bae9de848c8dfff7271302ee748a554f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
576b578b9b1f7993e8ac4c0d02b349b8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1c8d733e97566147a94100b0664bf84b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
6c1e72c947c8b80c7df0a6eb736e09f4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [ ]:
pd.concat([taxa,asv],axis=1, join='inner') 

In [48]:
asv

,10,11,12,13,14,17,18,19,2,20,...,27,29,3,30,4,5,6,7,8,9
feature_ID,,,,,,,,,,,,,,,,,,,,,
75eb829cc54be1d37541bad22e2b104c,0.0,6864.0,3937.0,10355.0,0.0,52431.0,44.0,10142.0,1280.0,13208.0,...,24.0,2.0,18.0,33.0,10272.0,42.0,0.0,2144.0,129.0,5270.0
aad49886d464a811b3c71c4fff2be6a6,38.0,51771.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1307.0,0.0,0.0,0.0,0.0,0.0,563.0,273.0,35268.0
fc4acd2822bc79e4a208ca9e00d17dbe,704.0,517.0,9050.0,100.0,0.0,258.0,3174.0,11954.0,2380.0,8907.0,...,152.0,1867.0,173.0,730.0,757.0,17924.0,34.0,1175.0,2275.0,74.0
0757a94ad3aae3e0b6744a115e0124c5,14413.0,112.0,1224.0,3476.0,0.0,0.0,2237.0,342.0,953.0,2577.0,...,250.0,2218.0,1838.0,568.0,8699.0,13664.0,2498.0,847.0,476.0,206.0
637cffe4d351e63641b7e01fb94e49f9,0.0,4155.0,0.0,0.0,6088.0,7772.0,6150.0,1060.0,57.0,1126.0,...,15495.0,0.0,1104.0,0.0,0.0,0.0,0.0,0.0,549.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bae9de848c8dfff7271302ee748a554f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
576b578b9b1f7993e8ac4c0d02b349b8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1c8d733e97566147a94100b0664bf84b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [46]:
taxa.index.name = "feature_ID"
asv.index.name = "feature_ID"
taxa_asv = pd.concat([taxa,asv],axis=1, join='inner') 

In [49]:
taxa_asv = pd.concat([taxa,asv],axis=1, join='outer') 

In [56]:
taxa_asv

taxa_asv[taxa_asv.k.isna()].iloc[:, 7:].sum(axis=1)

feature_ID
59e859470d17c4cda77ecf3d24a5ca32    14833.0
2d40054a9e3e25f867043482aa14f109    11370.0
3d4c4ff85bff56938b4e3f2b6ab8ff24    10088.0
92dafbcfa4eedf582e826f66429c527f     7767.0
3de87606bdb007771f7b38d48f01ec56     7162.0
                                     ...   
ac213190ff500a2afc8c0031bf69b458        2.0
2ef1a64fba95556cf8f0d3d6d4ccb198        2.0
8fb38b5909527da8bd0e5991e8d65611        2.0
576b578b9b1f7993e8ac4c0d02b349b8        2.0
6c1e72c947c8b80c7df0a6eb736e09f4        2.0
Length: 656, dtype: float64

In [57]:
import pandas as pd

class ASV(pd.DataFrame):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def custom_method(self):
        # 在这里编写您的自定义方法
        pass
    

In [62]:
class ASV(pd.DataFrame):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def custom_method(self):
        # 在这里编写您的自定义方法
        pass
    def to_relative(self):
        return self/self.sum(axis=0)
    
    def 

# 创建一个 DataFrame 对象
df = pd.DataFrame({'sample1': [10, 20, 30], 'sample2': [40, 50, 60]}, index=['asv1', 'asv2', 'asv3'])

# 转换为 ASV 对象
asv = ASV(df)

In [63]:
asv.to_relative()

,sample1,sample2
asv1,0.166667,0.266667
asv2,0.333333,0.333333
asv3,0.500000,0.400000


In [ ]:
asv.to_